# Rulefit demo - Titanic Dataset

## H2O Rulefit algorithm

Rulefit algorithm combines tree ensembles and linear models to take advantage of both methods: a tree ensemble accuracy and a linear model interpretability. The general algorithm fits a tree ensebmle to the data, builds a rule ensemble by traversing each tree, evaluates the rules on the data to build a rule feature set and fits a sparse linear model (LASSO) to the rule feature set joined with the original feature set.

For more information, refer to: http://statweb.stanford.edu/~jhf/ftp/RuleFit.pdf by Jerome H. Friedman and Bogden E. Popescu.

## Demo example

We will train a rulefit model to predict the rules defining whether or not someone will survive:


In [27]:
import h2o
from h2o.estimators import H2ORuleFitEstimator, H2ORandomForestEstimator

# init h2o cluster
h2o.init(strict_version_check=False, url="http://192.168.59.147:54321")

Checking whether there is an H2O instance running at http://192.168.59.147:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_231"; Java(TM) SE Runtime Environment (build 1.8.0_231-b11); Java HotSpot(TM) 64-Bit Server VM (build 25.231-b11, mixed mode)
  Starting server from /Users/zuzanaolajcova/IdeaProjects/h2o-3/build/h2o.jar
  Ice root: /var/folders/zn/5r1mf9597431rjrsg0lmr4tc0000gn/T/tmpjcltbl_r
  JVM stdout: /var/folders/zn/5r1mf9597431rjrsg0lmr4tc0000gn/T/tmpjcltbl_r/h2o_zuzanaolajcova_started_from_python.out
  JVM stderr: /var/folders/zn/5r1mf9597431rjrsg0lmr4tc0000gn/T/tmpjcltbl_r/h2o_zuzanaolajcova_started_from_python.err
  Server is running at http://127.0.0.1:54329
Connecting to H2O server at http://127.0.0.1:54329 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Europe/Prague
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.31.0.99999
H2O_cluster_version_age:,4 hours and 10 minutes
H2O_cluster_name:,H2O_from_python_zuzanaolajcova_9t2ii7
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.556 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"accepting new members, healthy"


In [28]:
df = h2o.import_file("https://s3.amazonaws.com/h2o-public-test-data/smalldata/gbm_test/titanic.csv",
                       col_types={'pclass': "enum", 'survived': "enum"})
x =  ["age", "sibsp", "parch", "sex", "pclass"]

# Split the dataset into train and test
train, test = df.split_frame(ratios=[.8], seed=1234)

Parse progress: |█████████████████████████████████████████████████████████| 100%


Using the `algorithm` parameter, a user can set whether algorithm will use DRF or GBM to fit a tree enseble. 

Using the `min_rule_length` and `max_rule_length` parameters, a user can set interval of tree enseble depths to be fitted. The bigger this interval is, the more tree ensembles will be fitted (1 per each depth) and the bigger the rule feature set will be.

Using the `max_num_rules` parameter, the maximum number of rules to return can be set.

Using the `model_type` parameter, the type of base learners in the enseble can be set.

Using the `rule_generation_ntrees` parameter, the number of trees for tree enseble can be set.

In [29]:
rfit = H2ORuleFitEstimator(algorithm="drf", 
                               min_rule_length=1, 
                               max_rule_length=10, 
                               max_num_rules=100, 
                               model_type="rules_and_linear",
                               rule_generation_ntrees=50,
                               seed=1234)
rfit.train(training_frame=train, x=x, y="survived")

rulefit Model Build progress: |███████████████████████████████████████████| 100%


The output for the Rulefit model includes:
    - model parameters
    - rule importences in tabular form
    - training and validation metrics of the underlying linear model

In [30]:
print(rfit.rule_importance())


Rule Importance: 


,,variable,coefficient,rule
0,,M2T21N13,1.298698e+00,"(sex in {female}) & (sibsp < 3.5 or sibsp is NA) & (pclass in {1, ..."
1,,M2T23N21,-8.455729e-01,"(sex in {male} or sex is NA) & (pclass in {2, 3} or pclass is NA) ..."
2,,M1T0N7,3.807125e-01,"(pclass in {1, 2}) & (sex in {female})"
3,,M1T28N10,-3.445493e-01,(sex in {male} or sex is NA) & (age >= 13.496771812438965 or age i...
4,,M1T23N7,3.310400e-01,(sex in {female}) & (sibsp < 2.5 or sibsp is NA)
5,,M1T37N10,-2.323243e-01,(sex in {male} or sex is NA) & (age >= 14.977890968322754 or age i...
6,,M4T3N45,-2.772966e-02,"(sex in {male} or sex is NA) & (pclass in {2, 3} or pclass is NA) ..."
7,,M1T1N7,1.631369e-13,"(pclass in {1, 2}) & (sex in {female})"
8,,M1T35N9,-1.029435e-13,(sex in {male} or sex is NA) & (age >= 13.496771812438965 or age i...


There are several rules that can be recapped as:

### Higgest Likelihood of Survival:
1. women in class 1 or 2 with 3 siblings/spouses aboard or less
2. women in class 1 or 2
3. women with 2 siblings/spouses aboard or less

### Lowest Likelihood of Survival:
1. male in class 2 or 3 of age >= 9.4
2. male of age >= 13.4
3. male of age >= 14.8
4. male in class 2 or 3 with no parents/children aboard of age between 14 to 61

Note: The rules are additive. That means that if a passenger is described by multiple rules, their probability is added together from those rules.

## Accuracy comparision with underlying tree ensebmle

In [37]:
rfit.model_performance(test)


ModelMetricsBinomialGLM: rulefit
** Reported on test data. **

MSE: 0.14668210909176724
RMSE: 0.3829910039305979
LogLoss: 0.46163352928872214
Null degrees of freedom: 1053
Residual degrees of freedom: 1044
Null deviance: 1405.0919048764017
Residual deviance: 973.1234797406264
AIC: 993.1234797406264
AUC: 0.8361042692939246
AUCPR: 0.7904193564939762
Gini: 0.6722085385878491

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.44133306750339485: 


,,0,1,Error,Rate
0,0,526.0,122.0,0.1883,(122.0/648.0)
1,1,106.0,300.0,0.2611,(106.0/406.0)
2,Total,632.0,422.0,0.2163,(228.0/1054.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.441333,0.724638,3.0
1,max f2,0.160004,0.783832,7.0
2,max f0point5,0.809020,0.774478,1.0
3,max accuracy,0.523804,0.790323,2.0
4,max precision,0.809020,0.919048,1.0
5,max recall,0.156312,1.000000,8.0
6,max specificity,0.855041,0.973765,0.0
7,max absolute_mcc,0.523804,0.550968,2.0
8,max min_per_class_accuracy,0.441333,0.738916,3.0
9,max mean_per_class_accuracy,0.441333,0.775322,3.0



Gains/Lift Table: Avg response rate: 38.52 %, avg score: 38.52 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.195446,0.855041,2.381821,2.381821,0.917476,0.855041,0.917476,0.855041,0.465517,0.465517,138.182123,138.182123,0.439283
1,2,0.348197,0.523804,1.402839,1.952350,0.540373,0.530890,0.752044,0.712838,0.214286,0.679803,40.283940,95.234963,0.539371
2,3,0.400380,0.414533,1.132826,1.845540,0.436364,0.441333,0.710900,0.677453,0.059113,0.738916,13.282579,84.553965,0.550645
3,4,0.528463,0.307331,0.788433,1.589329,0.303704,0.307331,0.612208,0.587746,0.100985,0.839901,-21.156723,58.932883,0.506568
4,5,1.000000,0.156312,0.339525,1.000000,0.130785,0.158208,0.385199,0.385203,0.160099,1.000000,-66.047517,0.000000,0.000000


In [38]:
drf = H2ORandomForestEstimator(distribution="AUTO",ntrees=50, max_depth=10)
drf.train(x=x,y="survived", training_frame=train)
drf.model_performance(test)

drf Model Build progress: |███████████████████████████████████████████████| 100%

ModelMetricsBinomial: drf
** Reported on test data. **

MSE: 0.14447151328039276
RMSE: 0.3800940847742737
LogLoss: 0.45394564696662937
Mean Per-Class Error: 0.2181181445751288
AUC: 0.8467027884234175
AUCPR: 0.8142832719617603
Gini: 0.693405576846835

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5301614665985108: 


,,0,1,Error,Rate
0,0,149.0,12.0,0.0745,(12.0/161.0)
1,1,34.0,60.0,0.3617,(34.0/94.0)
2,Total,183.0,72.0,0.1804,(46.0/255.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.530161,0.722892,59.0
1,max f2,0.118173,0.797491,131.0
2,max f0point5,0.530161,0.785340,59.0
3,max accuracy,0.530161,0.819608,59.0
4,max precision,0.999048,1.000000,0.0
5,max recall,0.013437,1.000000,164.0
6,max specificity,0.999048,1.000000,0.0
7,max absolute_mcc,0.530161,0.604202,59.0
8,max min_per_class_accuracy,0.327000,0.744681,85.0
9,max mean_per_class_accuracy,0.530161,0.781882,59.0



Gains/Lift Table: Avg response rate: 36.86 %, avg score: 36.06 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.011765,0.997188,2.712766,2.712766,1.000000,0.998254,1.000000,0.998254,0.031915,0.031915,171.276596,171.276596,0.031915
1,2,0.023529,0.994449,2.712766,2.712766,1.000000,0.995806,1.000000,0.997030,0.031915,0.063830,171.276596,171.276596,0.063830
2,3,0.031373,0.992747,2.712766,2.712766,1.000000,0.993585,1.000000,0.996169,0.021277,0.085106,171.276596,171.276596,0.085106
3,4,0.043137,0.989720,2.712766,2.712766,1.000000,0.991489,1.000000,0.994893,0.031915,0.117021,171.276596,171.276596,0.117021
4,5,0.050980,0.986592,2.712766,2.712766,1.000000,0.987918,1.000000,0.993820,0.021277,0.138298,171.276596,171.276596,0.138298
5,6,0.101961,0.933310,2.712766,2.712766,1.000000,0.967722,1.000000,0.980771,0.138298,0.276596,171.276596,171.276596,0.276596
6,7,0.152941,0.794459,2.295417,2.573650,0.846154,0.873828,0.948718,0.945123,0.117021,0.393617,129.541735,157.364975,0.381195
7,8,0.207843,0.646286,1.743921,2.354476,0.642857,0.693212,0.867925,0.878581,0.095745,0.489362,74.392097,135.447611,0.445883
8,9,0.301961,0.497019,1.695479,2.149074,0.625000,0.565879,0.792208,0.781115,0.159574,0.648936,69.547872,114.907433,0.549557
9,10,0.400000,0.344595,0.868085,1.835106,0.320000,0.429531,0.676471,0.694943,0.085106,0.734043,-13.191489,83.510638,0.529074
